In [27]:
import scipy.io as sio
import numpy as np
from scipy.stats import median_abs_deviation
from scipy.signal import find_peaks, filtfilt, iirnotch, savgol_filter

def filter_peaks(peaks:np.ndarray, peak_indices:np.ndarray, window:int, inverted:bool=False) -> np.ndarray:
    if inverted:
        peaks = -1*peaks
    true_peak_indices = []
    i = 0
    while i<len(peak_indices):
        cluster = [[peaks[i], peak_indices[i]]]
        if i<len(peak_indices)-1:
            while peak_indices[i+1]-peak_indices[i] < window:
                cluster.append([peaks[i+1], peak_indices[i+1]])
                i = i+1
                if i == len(peak_indices)-1:
                    break
        true_peak_indices.append(max(cluster)[1])
        i = i+1
    true_peak_indices = np.array(true_peak_indices)
    return true_peak_indices

def _find_peaks(ecg:np.ndarray, min_height:float, max_height:float=None, inverted:bool=False) -> np.ndarray:
    sign = 1
    if inverted:
        sign = -1
    if max_height:
        height = (sign*min_height, sign*max_height)
    else:
        height = sign*min_height
    peak_indices, _ = find_peaks(sign*ecg, height=height)
    return peak_indices

def reject_outliers(data, m = 2.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    return data[s<m]

def get_QRS(ekg, factor=5, margin=100, fs=500):
    b, a = iirnotch(0.05 , Q=0.005, fs=fs)
    bw_fixed_ecg = filtfilt(b, a, ekg[1])
    median = np.median(bw_fixed_ecg)
    bw_fixed_ecg = bw_fixed_ecg-median

    # median absolute deviation (kind of like standard deviation but from the median rather than the mean)
    mad = median_abs_deviation(bw_fixed_ecg)

    # find Rs
    R_indices, _ = find_peaks(bw_fixed_ecg, height=factor*mad)
    Rs = bw_fixed_ecg[R_indices]

    # fitler Rs
    window = 20
    true_R_indices = []
    i = 0
    for i in range(len(R_indices)):
        cluster = [[Rs[i], R_indices[i]]]
        while i < len(R_indices)-1 and R_indices[i+1]-R_indices[i] < window:
            cluster.append([Rs[i+1], R_indices[i+1]])
            i += 1
        true_R_indices.append(max(cluster)[1])
    true_R_indices = np.array(true_R_indices)

    # deciding the cutoff for S
    S_factor = 4
    S_cutoff = -S_factor * mad
    cut_ecg = bw_fixed_ecg[true_R_indices[1]+margin:true_R_indices[-2]-margin]

    # finding and filtering Ss
    S_indices = _find_peaks(cut_ecg, min_height=S_cutoff, inverted=True)
    Ss = cut_ecg[S_indices]
    true_S_indices = filter_peaks(Ss, S_indices, window=100, inverted=True)

    # deciding the cutoff for Q
    Q_factor = 4
    Q_cutoff = -Q_factor*mad

    # finding and filtering Qs
    Q_indices = _find_peaks(cut_ecg, min_height=Q_cutoff, max_height=S_cutoff, inverted=True)
    Qs = cut_ecg[Q_indices]
    true_Q_indices = filter_peaks(Qs, Q_indices, window=50, inverted=True)

    # average QRS interval

    # find first zero crossing before Q index
    Q_zero_crossings = []
    for i in true_Q_indices:
        for j in range(i, -1, -1):
            if cut_ecg[j] <= 0:
                Q_zero_crossings.append(i)
                break
    Q_zero_crossings = np.array(Q_zero_crossings)

    # find first zero crossing after S index
    S_zero_crossings = []
    for i in true_S_indices:
        for j in range(i, len(cut_ecg)-1):
            if cut_ecg[j] >= 0:
                S_zero_crossings.append(i)
                break
    S_zero_crossings = np.array(S_zero_crossings)

    # average QRS interval
    print(S_zero_crossings)
    print(Q_zero_crossings)
    qrs_intervals = S_zero_crossings - Q_zero_crossings
    qrs_intervals = reject_outliers(qrs_intervals, 3)
    avg_qrs_interval = np.mean(qrs_intervals)

In [6]:
data = sio.loadmat('training/chapman_shaoxing/g5/JS04193.mat')['val']

(12, 5000)

In [28]:
get_QRS(data)

[]
[  43  181  427  602  913 1261 1431 1589 1763 1989 2173 2455 2721 3010
 3354 3536 3718 3889 4033 4173 4334]
[]


ValueError: operands could not be broadcast together with shapes (21,) (0,) 

In [9]:
get_QRS(data)

[  48  187  433  606  920 1265 1436 1595 1770 1992 2180 2461 2725 3016
 3361 3543 3724 3894 4038 4179 4339]
[]


ValueError: operands could not be broadcast together with shapes (21,) (0,) 